In [1]:
a = "a"

In [6]:
a

'a'

In [7]:
b = "foo"

In [9]:
b = ""

def t():
    global b
    b = "c"

t()
b

'c'

In [11]:
"""Simple CRDs from python classes"""

from __future__ import annotations
from typing import Dict, Any, List
import os

from pydantic.dataclasses import dataclass



@dataclass
class GVK:
    """Group Version Kind"""

    group: str
    """API group e.g. foo.bar.com"""

    version: str
    """API version e.g. v1alpha1"""

    kind: str
    """Kind of resource"""

    def group_version(self) -> str:
        """group/version from the gvk e.g. foo.bar.com/v1alpha1

        Returns:
            str: A Kubernetes formatted group/version
        """
        return os.path.join(self.group, self.version)

    @classmethod
    def from_group_version(cls, gv: str, kind: str):
        group, version = cls.parse_group_version(gv)
        return cls(group=group, version=version, kind=kind)

    @classmethod
    def from_resource(cls, resource: Dict[str, Any]):
        """create a GVK from an existing resource

        Args:
            resource (Dict[str, Any]): a Kubernetes resource
        """
        group_version = resource["apiVersion"]
        group, version = cls.parse_group_version(group_version)
        return cls(group=group, version=version, kind=resource["kind"])

    @classmethod
    def parse_group_version(cls, gv: str) -> List[str]:
        return gv.split("/")


In [12]:
GVK(group="g", version="v", kind="k")

GVK(group='g', version='v', kind='k')

In [13]:
GVK.from_group_version("g.a/v1", "Test")

GVK(group='g.a', version='v1', kind='Test')

In [14]:
from dataclasses_jsonschema import JsonSchemaMixin

In [16]:
@dataclass
class Te(JsonSchemaMixin):
    a: str = "aaa"
    b: int = 2

Te.json_schema()

{'type': 'object',
 'properties': {'a': {'type': 'string', 'default': 'aaa'},
  'b': {'type': 'integer', 'default': 2}},
 'description': "Te(a: 'str' = 'aaa', b: 'int' = 2)",
 '$schema': 'http://json-schema.org/draft-06/schema#'}

In [18]:
import time
def nextSquare():
    i = 1
  
    # An Infinite loop to generate squares 
    while True:
        time.sleep(1)
        yield i*i                
        i += 1  # Next execution resumes 
                # from this point     
  
# Driver code to test above generator 
# function
for num in nextSquare():
    if num > 100:
         break    
    print(num)

1
4
9
16
25
36
49
64
81
100


In [19]:
a: List[str] = ["abc", "efg", "hij"]

In [20]:
f"a: {a}"

"a: ['abc', 'efg', 'hij']"

In [21]:
key = "hello"
value = "cruel world"
f'LABEL "{key}"="{value}"'

'LABEL "hello"="cruel world"'

In [22]:
import git

In [30]:
repo = git.Repo(".", search_parent_directories=True)
repo.working_dir

'/Users/patrickbarker/artificalcreatures/arc'

In [26]:
repo.untracked_files  

['Dockerfile',
 'Dockerfile.arc',
 'Makefile',
 'arc/config.py',
 'arc/image/__init__.py',
 'arc/image/build.py',
 'arc/image/build_test.py',
 'arc/image/file.py',
 'arc/image/file_test.py',
 'arc/image/hash.py',
 'arc/image/registry.py',
 'arc/kube/resource_test.py',
 'arc/kube/run.py',
 'arc/kube/run_test.py',
 'git_dif_stat.txt']

In [33]:
dirty = ""
for untracked in repo.untracked_files:
    with open(os.path.join(repo.working_dir, untracked), "r") as f:
        dirty += f.read()




In [41]:
t = repo.head.commit.tree
diff = repo.git.diff(t)

In [38]:
import hashlib

In [39]:
h = hashlib.new("sha256")

In [43]:
h.update(str.encode(diff))

In [45]:
h.hexdigest()

'a6c29a9901667ac68163d1f43005b5d3e79cd7edf4802e738918ee12ea6ef0b9'

In [46]:
from docker import APIClient
cli = APIClient(base_url="unix:///var/run/docker.sock")

In [51]:
cli.images()

[{'Containers': -1,
  'Created': 1643515505,
  'Id': 'sha256:77a3c889e1e61c3e706a59de789bfb400b609cc2f488cbbb6006163fe893c52d',
  'Labels': None,
  'ParentId': 'sha256:61f35ead7d2e9c2e5281cca29c3aef3026732b25f3732a0020cf015c40b67843',
  'RepoDigests': ['oldoceancreature/arc@sha256:e3468cfba60305848b4223d0ce0bf95f8dcc584a040a3d17ae412411f353dd86'],
  'RepoTags': ['oldoceancreature/arc:be9fba39bc4cccfb4ad0c56d9f8fe12deb19b4ce-1643515470'],
  'SharedSize': -1,
  'Size': 1755600057,
  'VirtualSize': 1755600057},
 {'Containers': -1,
  'Created': 1643514962,
  'Id': 'sha256:03798e1b5f81074e087b85e9c9bdfdca4099f71423986b1ffae1274b8202093f',
  'Labels': None,
  'ParentId': 'sha256:61f35ead7d2e9c2e5281cca29c3aef3026732b25f3732a0020cf015c40b67843',
  'RepoDigests': ['oldoceancreature/arc@sha256:bcf7ba0367eef676beb5e91adcdfd81a69488416507b739220976aafbad970fe'],
  'RepoTags': ['oldoceancreature/arc:be9fba39bc4cccfb4ad0c56d9f8fe12deb19b4ce-1643514927'],
  'SharedSize': -1,
  'Size': 1755600221,
  

In [53]:
from kubernetes.client import V1PersistentVolumeClaim, V1PersistentVolumeClaimSpec, V1Pod, V1PodSpec
from kubernetes import client, config
config.load_kube_config()
core_v1_api = client.CoreV1Api()

In [80]:
from kubernetes.client.rest import ApiException
try:
    core_v1_api.read_namespaced_pod("default", "abc")
except ApiException as e:
    if e.status == 404:
        print("yea")
    print(e.status)

yea
404


In [58]:
repo = git.Repo(search_parent_directories=True)
g = repo.git

In [60]:
g.ls_files(".")

'.gitignore\n.pylintrc\nDockerfile\nDockerfile.arc\nLICENSE\nMakefile\nREADME.md\narc/__init__.py\narc/config.py\narc/data/__init__.py\narc/data/controller.py\narc/data/dataset.py\narc/data/dataset_test.py\narc/data/s3.py\narc/image/__init__.py\narc/image/build.py\narc/image/build_test.py\narc/image/client.py\narc/image/file.py\narc/image/file_test.py\narc/image/id.py\narc/image/registry.py\narc/kube/__init__.py\narc/kube/client.py\narc/kube/controller.py\narc/kube/entrypoint.py\narc/kube/resource.py\narc/kube/resource_test.py\narc/kube/run.py\narc/kube/run_test.py\narc/kube/test.ipynb\narc/model/classifier/base.py\narc/model/classifier/multiclass.py\narc/multiclass.py\narc/train.py\narc/xy.py\ngit_dif_stat.txt\npoetry.lock\npyproject.toml\ntmp/test_crd.yaml\ntmp/test_obj.json\ntmp/test_obj1.json'

In [71]:
from pathlib import Path
def list_paths(root_tree, path=Path(".")):
    for blob in root_tree.blobs:
        yield path / blob.name
    for tree in root_tree.trees:
        yield from list_paths(tree, path / tree.name)

all_files = repo.untracked_files
for path in list_paths(repo.tree()):
    all_files.append(str(path))
all_files



['test.y',
 '.gitignore',
 '.pylintrc',
 'Dockerfile',
 'Dockerfile.arc',
 'LICENSE',
 'Makefile',
 'README.md',
 'git_dif_stat.txt',
 'poetry.lock',
 'pyproject.toml',
 'arc/__init__.py',
 'arc/config.py',
 'arc/multiclass.py',
 'arc/train.py',
 'arc/xy.py',
 'arc/data/__init__.py',
 'arc/data/controller.py',
 'arc/data/dataset.py',
 'arc/data/dataset_test.py',
 'arc/data/s3.py',
 'arc/image/__init__.py',
 'arc/image/build.py',
 'arc/image/build_test.py',
 'arc/image/client.py',
 'arc/image/file.py',
 'arc/image/file_test.py',
 'arc/image/id.py',
 'arc/image/registry.py',
 'arc/kube/__init__.py',
 'arc/kube/client.py',
 'arc/kube/controller.py',
 'arc/kube/entrypoint.py',
 'arc/kube/resource.py',
 'arc/kube/resource_test.py',
 'arc/kube/run.py',
 'arc/kube/run_test.py',
 'arc/kube/test.ipynb',
 'arc/model/classifier/base.py',
 'arc/model/classifier/multiclass.py',
 'tmp/test_crd.yaml',
 'tmp/test_obj.json',
 'tmp/test_obj1.json']

In [1]:
def foo():
    print("Bar")

import inspect

mod = inspect.getmodule(foo)
fil = inspect.getfile(foo)
mod
fil

'/var/folders/2y/6b7m4pk176d8t6hmvwhj93880000gn/T/ipykernel_4633/238862785.py'

In [2]:

def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False 

In [4]:
isnotebook()

True

In [5]:
get_ipython()

In [76]:
from arc.kube.entrypoint import generate_entrypoint

generate_entrypoint(foo)

'# This module is used as the entrypoint for the container\nfrom __main__ import foo\nfrom xmlrpc.server import SimpleXMLRPCServer\nfrom xmlrpc.server import SimpleXMLRPCRequestHandler\n\n# Restrict to a particular path.\nclass RequestHandler(SimpleXMLRPCRequestHandler):\n    rpc_paths = ("/RPC2",)\n\n# Create server\nwith SimpleXMLRPCServer(("localhost", 8000),\n                        requestHandler=RequestHandler) as server:\n\n    # register the function\n    server.register_function(foo)\n\n    # Run the server\'s main loop\n    server.serve_forever()\n'

In [77]:
import os
os.path.join("/app/", "/")

'/'

In [78]:
from kubernetes.client import CoreV1Api, V1Pod, V1PodSpec, V1Container, V1ObjectMeta, V1ContainerPort, V1Namespace
from kubernetes.client.rest import ApiException
from kubernetes import config

In [79]:
config.load_kube_config()
core_v1_api = CoreV1Api()

ConfigException: File does not exist: /var/folders/2y/6b7m4pk176d8t6hmvwhj93880000gn/T/tmpdbopuw6q

In [9]:
from kubernetes.client import CoreV1Api, V1Pod, V1PodSpec, V1Container, V1ObjectMeta, V1ContainerPort, V1Namespace
from kubernetes.client.rest import ApiException
from kubernetes import config
from kubernetes.stream import portforward
import socket
import xmlrpc.client

config.load_kube_config()
core_v1_api = CoreV1Api()

socket_create_connection = socket.create_connection

def kubernetes_create_connection(address, *args, **kwargs):
    print("address: ", address)
    dns_name = address[0]
    if isinstance(dns_name, bytes):
        dns_name = dns_name.decode()
    dns_name = dns_name.split(".")
    if dns_name[-1] != "kubernetes":
        return socket_create_connection(address, *args, **kwargs)
    if len(dns_name) not in (3, 4):
        raise RuntimeError("Unexpected kubernetes DNS name.")
    namespace = dns_name[-2]
    name = dns_name[0]
    port = address[1]
    pf = portforward(core_v1_api.connect_get_namespaced_pod_portforward, name, namespace, ports=str(port))
    return pf.socket(port)

socket.create_connection = kubernetes_create_connection


In [22]:
####
server_addr = f"http://fn-main-foo-2a26cff-c0f711a.pod.arc-arc.kubernetes:8000"
print("connecting to: ", server_addr)
with xmlrpc.client.ServerProxy(server_addr) as proxy:
    print("connected")
    res = None
    cmd = f"res = proxy.foo()"
    d = {}
    exec(cmd, locals(), d)
    print("d: ", d)
    print(res)

connecting to:  http://fn-main-foo-2a26cff-c0f711a.pod.arc-arc.kubernetes:8000
connected
address:  ('fn-main-foo-2a26cff-c0f711a.pod.arc-arc.kubernetes', 8000)
d:  {'res': 'hello world'}
None


In [25]:
from arc.config import RemoteSyncStrategy

a = RemoteSyncStrategy.CONTAINER

a.value

f"val {a.value}"

'val container'

In [27]:
if "container" == "contaier":
    print("hellpo")

In [1]:
# Test Apply
from kubernetes.client import CoreV1Api, V1Pod, V1PodSpec, V1Container, V1ObjectMeta, V1ContainerPort, V1Namespace
from kubernetes.client.rest import ApiException
from kubernetes import config

config.load_kube_config()
core_v1_api = CoreV1Api()


In [5]:
pod = V1Pod(
    metadata=V1ObjectMeta(
        name="tester",
        labels={
            "test": "yes"
        },
    ),
    spec=V1PodSpec(containers=[V1Container(name="test", image="ubuntu", command=["sleep", "infinity"])]),
)
core_v1_api.patch_namespaced_pod("tester", "default", pod, content_type="application/apply-patch+yaml")

ApiTypeError: Got an unexpected keyword argument 'content_type' to method patch_namespaced_pod

In [10]:
from tempfile import TemporaryFile
import tarfile

with tarfile.open("./tmp/test.tar", mode="w") as tar:
    tar.add("./sync.py", "./tmp/a")

In [17]:
from typing import Union, List
def a(b: Union[str, List[str]]):
    if isinstance(b, List):
        print("list!")
    else:
        print("no list")

a(["test"])

list!


In [23]:
a = "t/est/"
a.lstrip("/")

't/est/'

In [ ]:
""